<a href="https://www.kaggle.com/code/dascient/cryptofore-arima-forecasting-approach?scriptVersionId=216324944" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Welcome to our Universe
## [@donutz.ai](www.donutz.ai/#)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

from IPython.display import clear_output
clear_output()
import time,csv
import matplotlib.pyplot as plt
from dateutil.tz import tzlocal
from datetime import datetime
import seaborn as sns
from PIL import Image
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        clear_output()

# could've done this better, but hey...
# for kaggle
# rootdir = '/kaggle/input/g-research-crypto-forecasting/'

# for home
rootdir = '/kaggle/input/g-research-crypto-forecasting/'

# data pull
example_sample_submission = pd.read_csv(rootdir+'example_sample_submission.csv')
asset_details = pd.read_csv(rootdir+'asset_details.csv')
example_test = pd.read_csv(rootdir+'example_test.csv')
train = pd.read_csv(rootdir+'train.csv')
supplemental_train = pd.read_csv(rootdir+'supplemental_train.csv')

print('csv/files loaded')
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# timestamping 
supplemental_train['timestamp'] = supplemental_train.timestamp.astype('datetime64[s]')
example_test['timestamp'] = example_test.timestamp.astype('datetime64[s]')
train['timestamp'] = train.timestamp.astype('datetime64[s]')
print('timestamping complete')

In [ ]:
import numpy
import matplotlib.pyplot as plt
import math
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [ ]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

# fix random seed for reproducibility
numpy.random.seed(7)

In [ ]:
# load the dataset
# isolate to only one asset, for now...
# this also assumes only one variable - & we do not expect to output a complete "submission.csv"
dataframe = supplemental_train[supplemental_train.Asset_ID == 8][['Close']].reset_index(drop=True).head(100)

dataset = dataframe.values
dataset = dataset.astype('float32')

In [ ]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

In [ ]:
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

# reshape into X=t and Y=t+1
look_back = 3
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = numpy.reshape(testX, (testX.shape[0], testX.shape[1], 1))

 # ARIMA Model

In [ ]:
# Influence: https://machinelearningmastery.com/grid-search-arima-hyperparameters-with-python/

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from matplotlib import pyplot
# load dataset
# dataframe

# fit model
model = ARIMA(dataframe, order=(5,1,0))
model_fit = model.fit()
# summary of fit model
print(model_fit.summary())
# line plot of residuals
residuals = pd.DataFrame(model_fit.resid)
residuals.plot()
pyplot.show()
# density plot of residuals
residuals.plot(kind='kde')
pyplot.show()
# summary stats of residuals
print(residuals.describe())

In [ ]:
# grid search ARIMA parameters for time series
import warnings
from math import sqrt
from pandas import read_csv
from pandas import datetime
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(X, arima_order):
	# prepare training dataset
	train_size = int(len(X) * 0.66)
	train, test = X[0:train_size], X[train_size:]
	history = [x for x in train]
	# make predictions
	predictions = list()
	for t in range(len(test)):
		model = ARIMA(history, order=arima_order)
		model_fit = model.fit()
		yhat = model_fit.forecast()[0]
		predictions.append(yhat)
		history.append(test[t])
	# calculate out of sample error
	rmse = sqrt(mean_squared_error(test, predictions))
	return rmse

# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
	dataset = dataset.astype('float32')
	best_score, best_cfg = float("inf"), None
	for p in p_values:
		for d in d_values:
			for q in q_values:
				order = (p,d,q)
				try:
					rmse = evaluate_arima_model(dataset, order)
					if rmse < best_score:
						best_score, best_cfg = rmse, order
					print('ARIMA%s RMSE=%.3f' % (order,rmse))
				except:
					continue
	print('Best ARIMA%s RMSE=%.3f' % (best_cfg, best_score))

# load dataset
series = dataframe.Close

# evaluate parameters
p_values = [0,1,2] # please use -> [0, 1, 2, 4, 6, 8, 10]
d_values = range(0, 3)
q_values = range(0, 3)
warnings.filterwarnings("ignore")
evaluate_models(series.values, p_values, d_values, q_values)

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
model = SARIMAX(dataframe.Close, exog=None, order=(0, 1, 1), seasonal_order=(0, 1, 1, 7)).fit(method='cg')
model.summary()

In [ ]:
help(model)

# Submission

In [ ]:
#import gresearch_crypto
#env = gresearch_crypto.make_env()

# Training data is in the competition dataset as usual
#train_df = pd.read_csv('/kaggle/input/g-research-crypto-forecasting/train.csv', low_memory=False)

# load model
#model.fit(train_df)
#model.fit(train_df)

#iter_test = env.iter_test()

#for (test_df, sample_prediction_df) in iter_test:
#    sample_prediction_df['Target'] = model.predict(test_df)
#    env.predict(sample_prediction_df)


# submission
#sample_prediction_df['Target'].to_csv('submission.csv', index=False)
print("project complete, for now.")

In [ ]:
# still need output: submission.csv

# to be continued...